In [1]:
import sys
import os
project_dir = os.path.dirname(os.getcwd())
sys.path.append(project_dir) #notebooks can't look into source unless you append the parent project directory to the path
print(project_dir)

/home/ubuntu/courtney/usal_echo


In [2]:
import sqlalchemy
import numpy as np
import pandas as pd

In [3]:
sys.path.append('../src')
from d00_utils.db_utils import *

/home/ubuntu/anaconda3/envs/usal_echo/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [4]:
io_views = dbReadWriteViews()
io_segmentation = dbReadWriteSegmentation()

#Instances to write masks for
instances_w_labels_test_downsampleby5_df = io_views.get_table('instances_w_labels_test_downsampleby5')  
instances_w_labels_test_downsampleby5_df['studyidk'] = str(instances_w_labels_test_downsampleby5_df['studyidk']) 
print(instances_w_labels_test_downsampleby5_df.dtypes)

sopinstanceuid    object
filename          object
instanceidk        int64
view              object
studyidk          object
dtype: object


In [5]:
instances_w_labels_test_downsampleby5_df.shape

(1025, 5)

In [ ]:
chords_by_volume_mask_df = io_views.get_table("chords_by_volume_mask")
#chords_by_volume_mask_df = int(chords_by_volume_mask_df['studyidk'])
print(chords_by_volume_mask_df.shape)
print(chords_by_volume_mask_df.dtypes)

(1626282, 17)
instanceidk             int64
indexinmglist           int64
chordsequence           int64
chordtype              object
x1coordinate          float64
y1coordinate          float64
chordlength           float64
x2coordinate          float64
y2coordinate          float64
interchorddistance    float64
studyidk               object
frame                  object
view                   object
chamber                object
cardio_moment          object
sopinstanceuid         object
instancefilename       object
dtype: object


In [7]:
group_df = chords_by_volume_mask_df.groupby(["instanceidk", "indexinmglist"]).agg(
        {
            "x1coordinate": list,
            "y1coordinate": list,
            "x2coordinate": list,
            "y2coordinate": list,
            "chamber": pd.Series.unique,
            "frame": pd.Series.unique,
            "view" : pd.Series.unique,
            "instancefilename" : pd.Series.unique,
            "studyidk" : pd.Series.unique
        }
    )

In [8]:
group_df.head()

x1coordinate  \
instanceidk indexinmglist                                                      
57164       0              [422.0, 408.7043904209, 383.18264488286104, 32...   
            1              [445.0, 406.617336371168, 354.98270447906503, ...   
58207       0              [350.578541245637, 393.272428782834, 381.88757...   
            1              [360.0, 402.051138189609, 380.153414568827, 34...   
102950      0              [302.0, 244.70478260869598, 238.836057023644, ...   

                                                                y1coordinate  \
instanceidk indexinmglist                                                      
57164       0              [306.0, 308.185087082729, 311.836528976572, 33...   
            1              [306.0, 325.03537696768797, 345.54073758519996...   
58207       0              [336.381591034356, 364.54138919718804, 357.032...   
            1              [347.0, 359.631119442957, 352.89335832887, 350...   
102950      0              [374.0, 363.26195652173897, 349.431675938804, ...   

                                                                x2coordinate  \
instanceidk indexinmglist                                                      
57164       0              [297.0, 426.667431561997, 422.381486742424, 41...   
            1              [312.0, 447.06858778625997, 448.0, 448.0, 446....   
58207       0              [282.063809967396, 423.656674208145, 434.86361...   
            1              [285.0, 440.0, 440.17644298866895, 442.1610559...   
102950      0              [342.0, 321.691810344828, 373.83403090792, 382...   

                                                                y2coordinate  \
instanceidk indexinmglist                                                      
57164       0              [120.0, 296.113150831991, 285.493221275253, 27...   
            1              [109.0, 297.725648854962, 282.742258883249, 26...   
58207       0              [172.76432231020001, 351.817986425339, 334.848...   
            1              [124.0, 346.86804932735396, 332.706241147309, ...   
102950      0              [128.0, 375.780172413793, 371.382566001288, 35...   

                          chamber frame view instancefilename studyidk  
instanceidk indexinmglist                                               
57164       0                  lv     8  A4C         6RYXZO20     2882  
            1                  lv    15  A4C         6RYXZO20     2882  
58207       0                  lv     1  A4C         7QVURND6     2966  
            1                  lv     8  A4C         7QVURND6     2966  
102950      0                  lv    17  A4C         FN8YYXBS     5898

In [9]:
group_df.shape

(77442, 9)

In [10]:
group_df = group_df.reset_index()

In [11]:
instance_ids = instances_w_labels_test_downsampleby5_df['instanceidk']

In [20]:
merge_df = group_df[group_df['instanceidk'].isin(instance_ids)]
print(merge_df.shape)

(1157, 11)


In [21]:
merge_df

,instanceidk,indexinmglist,x1coordinate,y1coordinate,x2coordinate,y2coordinate,chamber,frame,view,instancefilename,studyidk
1659,779540,0,"[485.0, 480.90687919463096, 440.86949019132993...","[380.0, 381.66359060402704, 384.966698958586, ...","[396.0, 522.358037931813, 521.074113795439, 51...","[162.0, 364.740869835177, 352.222609505532, 33...",lv,51,A4C,1LHFYKEZ,43763
1660,779540,1,"[452.0, 421.94456521739096, 419.746739130435, ...","[349.0, 351.796739130435, 343.73804347826103, ...","[425.0, 482.456227436823, 483.36868231046896, ...","[187.0, 341.711462093863, 333.134386281588, 32...",lv,68,A4C,1LHFYKEZ,43763
1852,841585,0,"[437.0, 448.47381422924894, 384.99446280991697...","[416.0, 416.231027667984, 413.267314049587, 40...","[402.0, 491.178705211726, 503.536115635179, 51...","[181.0, 409.8707247557, 395.612174267101, 381....",lv,46,A4C,7SQZPLEJ,46919
1853,841585,1,"[414.0, 366.31768953068604, 360.64635761589403...","[405.0, 405.098916967509, 393.292715231788, 38...","[417.0, 412.328293736501, 461.257577036724, 46...","[177.0, 405.70431965442805, 394.616547066273, ...",lv,60,A4C,7SQZPLEJ,46919
2129,921901,0,"[508.0, 523.506117397454, 484.518352192362, 44...","[369.0, 365.96646039603996, 371.899381188119, ...","[417.0, 546.086449022875, 552.259347068626, 55...","[175.0, 355.37465535524905, 340.123966065748, ...",lv,51,A4C,EAKXA40T,50521
2130,921901,1,"[498.0, 517.637639664805, 472.912918994413, 44...","[351.0, 346.32437150837995, 356.97311452514, 3...","[427.0, 538.225681515957, 534.677044547872, 53...","[196.0, 336.893720079787, 328.681160239362, 32...",lv,68,A4C,EAKXA40T,50521
3462,1078635,0,"[448.0, 396.458362369337, 365.893518965934, 36...","[445.0, 445.746980255517, 435.379644841749, 42...","[432.0, 526.921595744681, 533.3222340425531, 5...","[202.0, 437.156808510638, 424.355531914894, 41...",lv,58,A4C,AFUJWQM9,57515
3463,1078635,1,"[451.0, 392.331048034934, 372.99314410480304, ...","[420.0, 427.60502183406095, 418.815065502183, ...","[436.0, 434.466129032258, 499.398387096774, 50...","[224.0, 424.38039828834803, 409.141194865043, ...",lv,75,A4C,AFUJWQM9,57515
3615,1084663,0,"[444.0, 474.72066794707604, 418.162003841229, ...","[477.0, 472.31379641485296, 480.941389244558, ...","[324.0, 502.87842102414197, 502.63526307242597...","[211.0, 459.61105066580103, 442.83315199740196...",lv,137,A2C,AKFY2K0G,57770
3616,1084663,1,"[431.0, 398.947803347281, 384.986204663212, 38...","[462.0, 471.10575313807504, 463.95401554404197...","[359.0, 474.048177676538, 472.14453302961306, ...","[252.0, 445.357053368044, 434.071160104133, 42...",lv,153,A2C,AKFY2K0G,57770


In [22]:
merge_df_idx = merge_df.set_index(['instanceidk', 'view', 'chamber'], inplace=False)

In [23]:
merge_df_idx

indexinmglist  \
instanceidk view chamber                  
779540      A4C  lv                   0   
                 lv                   1   
841585      A4C  lv                   0   
                 lv                   1   
921901      A4C  lv                   0   
...                                 ...   
3003810     A4C  la                   1   
                 lv                   2   
3004026     A4C  lv                   0   
                 lv                   1   
                 la                   2   

                                                               x1coordinate  \
instanceidk view chamber                                                      
779540      A4C  lv       [485.0, 480.90687919463096, 440.86949019132993...   
                 lv       [452.0, 421.94456521739096, 419.746739130435, ...   
841585      A4C  lv       [437.0, 448.47381422924894, 384.99446280991697...   
                 lv       [414.0, 366.31768953068604, 360.64635761589403...   
921901      A4C  lv       [508.0, 523.506117397454, 484.518352192362, 44...   
...                                                                     ...   
3003810     A4C  la       [410.0, 449.069, 455.207, 460.578034682081, 46...   
                 lv       [413.0, 422.05892370572195, 380.689414704934, ...   
3004026     A4C  lv       [469.0, 500.761822376009, 458.285467128028, 41...   
                 lv       [456.0, 477.70661415828397, 427.119842474853, ...   
                 la       [450.0, 496.69599609375, 498.08798828125, 499....   

                                                               y1coordinate  \
instanceidk view chamber                                                      
779540      A4C  lv       [380.0, 381.66359060402704, 384.966698958586, ...   
                 lv       [349.0, 351.796739130435, 343.73804347826103, ...   
841585      A4C  lv       [416.0, 416.231027667984, 413.267314049587, 40...   
                 lv       [405.0, 405.098916967509, 393.292715231788, 38...   
921901      A4C  lv       [369.0, 365.96646039603996, 371.899381188119, ...   
...                                                                     ...   
3003810     A4C  la       [377.0, 383.092, 391.276, 399.35043352601195, ...   
                 lv       [373.0, 374.158378746594, 369.819775878749, 36...   
3004026     A4C  lv       [414.0, 417.17958477508597, 413.53875432526, 4...   
                 lv       [393.0, 395.116449990511, 391.349349971532, 38...   
                 la       [399.0, 407.1759765625, 415.5279296875, 423.87...   

                                                               x2coordinate  \
instanceidk view chamber                                                      
779540      A4C  lv       [396.0, 522.358037931813, 521.074113795439, 51...   
                 lv       [425.0, 482.456227436823, 483.36868231046896, ...   
841585      A4C  lv       [402.0, 491.178705211726, 503.536115635179, 51...   
                 lv       [417.0, 412.328293736501, 461.257577036724, 46...   
921901      A4C  lv       [417.0, 546.086449022875, 552.259347068626, 55...   
...                                                                     ...   
3003810     A4C  la       [390.0, 404.15, 367.76666666666705, 358.023809...   
                 lv       [397.0, 450.726398601399, 456.782184599519, 46...   
3004026     A4C  lv       [415.0, 525.2361159929701, 531.70834797891, 53...   
                 lv       [417.0, 503.470344465332, 504.41103339599704, ...   
                 la       [408.0, 479.092883211679, 445.278649635037, 41...   

                                                               y2coordinate  \
instanceidk view chamber                                                      
779540      A4C  lv       [162.0, 364.740869835177, 352.222609505532, 33...   
                 lv       [187.0, 341.711462093863, 333.134386281588, 32...   
841585      A4C  lv       [181.0, 409.8707247557, 395.612174267101

In [26]:
merge_df_idx.groupby(level=1).count()

,indexinmglist,x1coordinate,y1coordinate,x2coordinate,y2coordinate,frame,instancefilename,studyidk
view,,,,,,,,
A2C,485,485,485,485,485,485,485,485
A4C,672,672,672,672,672,672,672,672
